In [14]:
import numpy as np
import pandas as pd

from helper.loader import load_ratings, load_movies, load_lists, load_500_1000_corr
from helper.preprocesser import preprocess_ratings, prepare_test_data_dense
from helper.application import application
import helper.collaborative as coll
import helper.showResults as sh

from tqdm import tqdm

# Performance

Notebook to check the performance of our different models by predicting ratings which we manually removed to have a ground truth

In [2]:
ratings = load_ratings()
movies = load_movies()
lists = load_lists()

In [17]:
ratings_new, lists_new = preprocess_ratings(ratings, lists, 500,1000)
dense_user_item = coll.get_dense_user_item(ratings_new)
popu_matrix = coll.get_popularity(lists_new, dense_user_item)

average_ratings = coll.compute_average_ratings(dense_user_item)

In [6]:
sum(dense_user_item.count())

3188952

starting with only 1/1000 of data missing, this represent 3188 ratings to predict

In [7]:
train_data,train_label = prepare_test_data_dense(dense_user_item,1/1000)

100%|██████████| 3188/3188 [00:00<00:00, 12684.96it/s]


In [19]:
train_label

{(1512, 2790): 3.0,
 (3199, 841): 4.0,
 (1161, 384): 3.0,
 (2333, 405): 5.0,
 (1870, 2432): 2.0,
 (931, 2009): 3.0,
 (2043, 1812): 3.0,
 (2836, 2453): 1.0,
 (4, 657): 5.0,
 (907, 1454): 4.0,
 (1383, 13): 4.0,
 (353, 2138): 3.0,
 (1764, 518): 4.0,
 (1958, 276): 4.0,
 (2537, 2506): 3.0,
 (2292, 119): 4.0,
 (484, 1557): 4.0,
 (3180, 1221): 4.0,
 (2175, 1150): 5.0,
 (3081, 2187): 2.0,
 (923, 2627): 1.0,
 (3340, 2787): 4.0,
 (3052, 1441): 2.0,
 (3311, 2712): 5.0,
 (3673, 1809): 4.0,
 (2427, 372): 5.0,
 (1304, 141): 4.0,
 (1662, 2830): 4.0,
 (2819, 2093): 4.0,
 (2942, 2471): 4.0,
 (3266, 126): 3.0,
 (1851, 893): 3.0,
 (1263, 2532): 3.0,
 (2708, 1664): 3.0,
 (3691, 52): 5.0,
 (3511, 1863): 4.0,
 (724, 2387): 4.0,
 (3005, 1035): 5.0,
 (1994, 2670): 5.0,
 (458, 2627): 2.0,
 (3046, 964): 3.0,
 (3426, 1680): 2.0,
 (740, 2209): 3.0,
 (1124, 921): 3.0,
 (1095, 748): 4.0,
 (1816, 1487): 3.0,
 (1282, 1267): 4.0,
 (2519, 1597): 3.0,
 (17, 1129): 4.0,
 (2963, 41): 4.0,
 (2636, 400): 5.0,
 (2814, 2599):

In [18]:
# Each removed ratings is for a new user -> will have to compute a lot of prediction, maybe something cleaner but didn't see

classic_norm_SSE = 0
classic_SSE = 0
hybrid_norm_SSE = 0
hybrid_SSE = 0
pop_norm_SSE = 0
pop_SSE = 0

rounded_classic_norm_error = 0
rounded_classic_error = 0
rounded_hybrid_norm_error = 0
rounded_hybrid_error = 0
rounded_pop_norm_error = 0
rounded_pop_error = 0


for user,item in tqdm(train_label.keys()):

    similarity = coll.get_k_dynamic_similar_users(dense_user_item,user,40)
    popularity = coll.get_k_popularity(popu_matrix,similarity)

    ##################
    weight_classic = similarity
    
    pred_norm = coll.predict_value_norm(user,item,weight_classic,dense_user_item,average_ratings)
    pred = coll.predict_value(user,item,weight_classic,dense_user_item,average_ratings)

    classic_norm_SSE += (pred_norm - train_label[(user,item)])**2
    classic_SSE += (pred - train_label[(user,item)])**2

    rounded_classic_norm_error += abs(round(pred_norm) - train_label[(user,item)])
    rounded_classic_error += abs(round(pred) - train_label[(user,item)])

    ##################
    hybrid = similarity.loc[user] + popularity.loc["popularity"]
    weight_hybrid = hybrid.to_frame().transpose() 

    pred_norm = coll.predict_value_norm(user,item,weight_hybrid,dense_user_item,average_ratings)
    pred = coll.predict_value(user,item,weight_hybrid,dense_user_item,average_ratings)

    hybrid_norm_SSE += (pred_norm - train_label[(user,item)])**2
    hybrid_SSE += (pred - train_label[(user,item)])**2

    rounded_hybrid_norm_error += abs(round(pred_norm) - train_label[(user,item)])
    rounded_hybrid_error += abs(round(pred) - train_label[(user,item)])

    ##################
    weight_pop = popularity

    pred_norm = coll.predict_value_norm(user,item,weight_pop,dense_user_item,average_ratings)
    pred = coll.predict_value(user,item,weight_pop,dense_user_item,average_ratings)

    pop_norm_SSE += (pred_norm - train_label[(user,item)])**2
    pop_SSE += (pred - train_label[(user,item)])**2

    rounded_pop_norm_error += abs(round(pred_norm) - train_label[(user,item)])
    rounded_pop_error += abs(round(pred) - train_label[(user,item)])
    ##################



  0%|          | 0/3188 [00:00<?, ?it/s]


KeyError: 1512

In [16]:
train_label[(1512,2790)]

3.0

In [15]:
for key1,key2 in train_label.keys():
    print(key2)

2790
841
384
405
2432
2009
1812
2453
657
1454
13
2138
518
276
2506
119
1557
1221
1150
2187
2627
2787
1441
2712
1809
372
141
2830
2093
2471
126
893
2532
1664
52
1863
2387
1035
2670
2627
964
1680
2209
921
748
1487
1267
1597
1129
41
400
2599
55
767
1297
464
1278
24
1753
2157
1152
16
1120
1358
511
1131
1395
2477
73
2392
1655
2616
1899
2220
1602
2181
404
833
1662
2258
1488
2520
1249
2579
89
1766
2395
2389
116
1346
325
893
1287
196
518
830
1050
1576
311
33
846
218
1250
136
883
2929
1135
834
153
623
1842
1366
710
702
937
259
1649
2163
939
197
30
2567
899
2225
302
900
2757
2391
983
657
625
2050
130
2480
1687
1845
2589
1183
2378
2599
2792
1898
2762
72
93
2900
2818
26
2787
141
383
568
185
1151
42
2610
1261
833
171
2612
67
1343
59
1270
9
974
307
836
96
1319
762
2636
275
1849
2859
1312
64
774
1303
898
185
1044
1035
2169
524
2566
1967
259
50
1759
1556
1904
1062
2735
2362
557
2892
44
194
2499
2528
562
1180
832
122
1728
567
2466
671
27
1459
345
2329
1035
940
2671
683
974
1943
1332
733
1089
1930
465
1